In [129]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import missingno as msno
import pandas_profiling
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
#解决中文显示问题
plt.rcParams["font.sans-serif"] = [u"SimHei"]
plt.rcParams["axes.unicode_minus"] = False
#解决图中负号问题
plt.rcParams['axes.unicode_minus']=False

In [130]:
column_name=['id','target','card_type','netlong','owner_city','arpu','out_prdct_fee','prdct_fee','point_fee','myth_fee','brand','listing_date','listing_price','nettime','user_level','sex','family_client','group_client','music','expirationtime','video_member','other_tv','finance','bank','campus_client','migu_tv','age','adis','arpu_avg','myth_fee_avg','out_prdct_fee_avg','prdct_fee_avg','os','battery_capacity','storage','screen_size']

# names = ['id','target','card_type','netlong','owner_city','arpu','out_prdct_fee','prdct_fee','point_fee','myth_fee','brand','listing_date','listing_price','nettime','user_level','sex','family_client','group_client','music','expirationtime','video_member','other_tv','finance','bank','campus_client','migu_tv','age','adis','arpu_avg','myth_fee_avg','out_prdct_fee_avg','prdct_fee_avg']
original = pd.read_table('ZZ0702GuangFa_Samples100000_FeatureData.txt',names=column_name)
display(original.head(),original.shape)

,id,target,card_type,netlong,owner_city,arpu,out_prdct_fee,prdct_fee,point_fee,myth_fee,...,age,adis,arpu_avg,myth_fee_avg,out_prdct_fee_avg,prdct_fee_avg,os,battery_capacity,storage,screen_size
0,13013302938,1,0.0,NaN,联通联通,9.0,0.0,0.0,0.0,0.0,...,NaN,NaN,38.863332,0.0,NaN,0.0,4.0,4120mAh,64GB,6.0
1,13022062073,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13035901978,1,NaN,NaN,NaN,37.2,0.0,0.0,0.0,0.0,...,NaN,NaN,37.013334,0.0,NaN,0.0,NaN,NaN,NaN,NaN
3,13042151031,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3050mAh,16/64GB,5.0
4,13045173577,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(103513, 36)

In [131]:
display(original.target.value_counts())

0    92989
1    10524
Name: target, dtype: int64

In [132]:
sample_false=original[original['target']==0].sample(n=10000, replace=False,random_state=0, axis=None).reset_index(drop=True)
sample_true=original[original['target']==1].sample(n=10000, replace=False,random_state=0, axis=None).reset_index(drop=True)
combin_df=sample_true.append(sample_false)
combin_df.to_excel('d:\sample_model.xlsx',index=None)

In [133]:
sample_true.head()

,id,target,card_type,netlong,owner_city,arpu,out_prdct_fee,prdct_fee,point_fee,myth_fee,...,age,adis,arpu_avg,myth_fee_avg,out_prdct_fee_avg,prdct_fee_avg,os,battery_capacity,storage,screen_size
0,18746191488,1,1.0,37.0,黑龙江哈尔滨,45.32,6.48,29.0,0.0,0.0,...,NaN,1.0,45.320000,0.0,6.48,29.0,NaN,NaN,NaN,NaN
1,13769713610,1,0.0,NaN,云南曲靖,115.70,0.00,0.0,0.0,0.0,...,NaN,NaN,115.099998,0.0,NaN,0.0,4.0,2920mAH,16GB/32GB,5.0
2,13838040676,1,1.0,NaN,河南郑州,143.20,0.20,135.0,9951.0,0.0,...,NaN,1.0,148.810003,0.0,0.20,135.0,4.0,3000mAh,32GB,5.5
3,15945389676,1,0.0,30.0,黑龙江牡丹江,60.99,1.00,90.0,0.0,0.0,...,NaN,NaN,60.990002,0.0,1.00,90.0,4.0,3000mAh,16GB,5.7
4,13888246777,1,1.0,NaN,云南昆明,258.00,0.00,0.0,0.0,0.0,...,NaN,NaN,258.000000,0.0,NaN,0.0,3.0,1715mAh,16GB/64GB/128GB,4.7


In [134]:
import random
def create_column_randomsample(column_random_dict):
    total_column_dict={}
    total_column_df=[]
    for columns_name in column_random_dict:
        if columns_name not in total_column_dict:
            total_column_dict[columns_name]=[]        
        for value_keys in column_random_dict[columns_name]:
            value_keys_counts=int(column_random_dict[columns_name][value_keys]*50000)
            for index in range(value_keys_counts):
                total_column_dict[columns_name].append(value_keys)
            
    result_df=pd.DataFrame(total_column_dict)
    for columns in result_df.columns:
        result_df[columns]=result_df[columns].sample(frac=1).reset_index(drop=True)
    return result_df

def create_number_randomsample(number_random_dict):
    import numpy as np
    
    total_column_dict={}
    total_column_df=[]
    for columns_name in number_random_dict:
        if columns_name not in total_column_dict:
            number_column_value=np.random.normal(loc=number_random_dict[columns_name]['mean'],scale=number_random_dict[columns_name]['std'],size=50000)
            if columns_name=='产品万份收益_基金产品':                
                total_column_dict[columns_name]=number_column_value.astype(np.float32)
            elif columns_name=='开户日期':
                total_column_dict[columns_name]=np.fabs(number_column_value).astype(np.float32)
            else:
                total_column_dict[columns_name]=number_column_value.astype(np.int32)
    return pd.DataFrame(total_column_dict)
            
            
True_column_random_dict={'性别':{'男':0.6,'女':0.4},'婚姻状况':{'已婚':0.65,'未婚':0.35},
                         '学历':{'小学':0.01,'初中':0.02,'高中':0.1,'初中中专':0.02,'高中中专':0.1,'大专':0.5,'本科':0.2,'研究生':0.05},
#                          '学位':{'其他':0.599,'学士':0.3,'硕士':0.1,'博士':0.001},
                         '征信认定年收入':{'5万':0.35,'10万':0.5,'20万':0.05,'30万':0.04,'40万':0.03,'50万':0.02,'大于50万':0.01},
                         '社会保障':{'1':0.7,'0':0.3},
                         '客户级信用额度':{'5000':0.05,'10000':0.15,'20000':0.3,'30000':0.3,'50000':0.2},
                         '美元账户额度':{'1':0.05,'0':0.95},
                         'CRM评级结果':{'普通':0.7,'准核心':0.25,'核心':0.05},
                         '客户评级结果':{'C1':0.15,'C2':0.15,'C3':0.3,'C4':0.2,'C5':0.15,'C6':0.05},
                         '借记卡层级':{'普卡':0.8,'金卡':0.13,'白金':0.05,'黑金':0.02},
                         '附卡卡数':{'0':0.6,'1':0.15,'2':0.1,'3':0.05,'4':0.05,'其他':0.05},
                         '存款期限':{'3个月':0.15,'6个月':0.15,'1年':0.3,'2年':0.2,'3年':0.15,'5年':0.05},
                         '是否为低保户':{'是':0.01,'否':0.99},
                         '受否贫困户':{'是':0.02,'否':0.98},
                         '行业':{'农林牧渔':0.06,'采矿业':0.14,'制造业':0.3,'住宿和餐饮业':0.2,'金融业':0.13,'房地产业':0.17},
                         '职业':{'公务员':0.15,'专业技术人员':0.35,'服务业人员':0.25,'商业人员':0.1,'农业生产人员':0.15},
                         '零售潜在客户':{'是':0.6,'否':0.4},
                         '理财产品风险等级':{'基本无风险':0.2,'较低风险':0.45,'中等风险':0.25,'高风险':0.1},
                         '手机品牌':{'苹果':0.3,'华为':0.2,'荣耀':0.1,'小米':0.1,'OPPO':0.16,'vivo':0.14},
                         '基金风险等级':{'A1':0.2,'A2':0.35,'A3':0.3,'A4':0.1,'A5':0.05},
                         '理财产品是否保本':{'是':0.7,'否':0.3},
                         '客户风险承受能力等级':{'较低':0.2,'基本':0.45,'中等':0.3,'较高':0.05},
                         '电子渠道购买标志':{'是':0.65,'否':0.35},
                        }
True_number_random_dict={'年龄':{'mean':30,'std':13.5},
                         '贷款放款本金':{'mean':500000,'std':300000},'账户余额':{'mean':250000,'std':150000},
                         '贷款期数':{'mean':30,'std':8},'剩余贷款本金':{'mean':300000,'std':40000},
                         '购买金额_事件表':{'mean':50000,'std':20000},'数量_事件表':{'mean':8,'std':3},
                         '可用余额_事件表':{'mean':70000,'std':30000},'分期金额_事件表':{'mean':20000,'std':10000},
                         '贷款期限_事件表':{'mean':9,'std':4},'未还本金_事件表':{'mean':9000,'std':3000},
                         '偿还金额_事件表':{'mean':6000,'std':3000},'金额_事件表':{'mean':12000,'std':9000},
                         '交易金额_交易日志表':{'mean':3000,'std':1000},'产品万份收益_基金产品':{'mean':0.9,'std':0.3},  
                         '车产价值_字典列表':{'mean':120000,'std':80000},'房产价值_字典列表':{'mean':1200000,'std':900000},
                         '交易金额_交易往来':{'mean':8000,'std':5000},'存期_交易账务':{'mean':16,'std':7},
                         '交易金额_交易账务':{'mean':9000,'std':4000},
                         '行内总资产_资产负债':{'mean':150000,'std':40000},'行内总负债_资产负债':{'mean':8000,'std':2500},
                         '本外币存款_资产负债':{'mean':10000,'std':5000},'人民币活期存款_资产负债':{'mean':50000,'std':10000},
                         '人民币定期存款_资产负债':{'mean':30000,'std':9000},'本外币理财_资产负债':{'mean':3000,'std':2000},
                         '人民币理财_资产负债':{'mean':90000,'std':28000},'万利宝理财_资产负债':{'mean':30000,'std':8000},
                         '保本型理财_人民币_资产负债':{'mean':80000,'std':24000},'结构性存款_人民币_资产负债':{'mean':40000,'std':9000},
                         '兴业宝资产_资产负债':{'mean':20000,'std':6000},'综合金融资产_资产负债':{'mean':15000,'std':4000},
                         '券商集合理财_资产负债':{'mean':9000,'std':1000},'保险_资产负债':{'mean':5000,'std':1000},
                         '个人贷款_资产负债':{'mean':75000,'std':6000},'个人购房按揭贷款_资产负债':{'mean':350000,'std':70000},
                         '个人委托贷款_资产负债':{'mean':15000,'std':4000},'个人消费贷款_资产负债':{'mean':5000,'std':800},
                         '开户日期':{'mean':3600,'std':4000},
                         '月收入_交易往来':{'mean':12000,'std':5000},
                         '月支出_交易往来':{'mean':4000,'std':1000},
                        }
                         

False_column_random_dict={'性别':{'男':0.45,'女':0.55},'婚姻状况':{'已婚':0.4,'未婚':0.6},
                         '学历':{'小学':0.02,'初中':0.03,'高中':0.1,'初中中专':0.1,'高中中专':0.4,'大专':0.3,'本科':0.03,'研究生':0.02},
#                          '学位':{'其他':0.599,'学士':0.3,'硕士':0.1,'博士':0.001},
                         '征信认定年收入':{'5万':0.55,'10万':0.35,'20万':0.04,'30万':0.03,'40万':0.02,'50万':0.008,'大于50万':0.002},
                         '社会保障':{'1':0.6,'0':0.4},
                         '客户级信用额度':{'5000':0.1,'10000':0.4,'20000':0.2,'30000':0.2,'50000':0.1},
                         '美元账户额度':{'1':0.01,'0':0.99},
                         'CRM评级结果':{'普通':0.83,'准核心':0.15,'核心':0.02},
                         '客户评级结果':{'C1':0.2,'C2':0.45,'C3':0.2,'C4':0.1,'C5':0.03,'C6':0.02},
                         '借记卡层级':{'普卡':0.9,'金卡':0.07,'白金':0.02,'黑金':0.01},
                         '附卡卡数':{'0':0.75,'1':0.1,'2':0.05,'3':0.05,'4':0.04,'其他':0.01},
                         '存款期限':{'3个月':0.1,'6个月':0.1,'1年':0.2,'2年':0.3,'3年':0.2,'5年':0.1},
                         '是否为低保户':{'是':0.05,'否':0.95},
                         '受否贫困户':{'是':0.07,'否':0.93},
                          
                         '行业':{'农林牧渔':0.25,'采矿业':0.2,'制造业':0.25,'住宿和餐饮业':0.15,'金融业':0.05,'房地产业':0.1},
                         '职业':{'公务员':0.05,'专业技术人员':0.15,'服务业人员':0.25,'商业人员':0.05,'农业生产人员':0.5},
                         '零售潜在客户':{'是':0.3,'否':0.7},
                         '理财产品风险等级':{'基本无风险':0.7,'较低风险':0.18,'中等风险':0.1,'高风险':0.02},
                         '手机品牌':{'苹果':0.1,'华为':0.1,'荣耀':0.3,'小米':0.3,'OPPO':0.1,'vivo':0.1},
                         '基金风险等级':{'A1':0.3,'A2':0.4,'A3':0.2,'A4':0.07,'A5':0.03},
                         '理财产品是否保本':{'是':0.85,'否':0.15},
                         '客户风险承受能力等级':{'较低':0.5,'基本':0.35,'中等':0.12,'较高':0.03},
                         '电子渠道购买标志':{'是':0.45,'否':0.55},
                        }

False_number_random_dict={'年龄':{'mean':23,'std':13.5},
                         '贷款放款本金':{'mean':200000,'std':100000},'账户余额':{'mean':80000,'std':50000.5},
                         '贷款期数':{'mean':35,'std':8},'剩余贷款本金':{'mean':200000,'std':80000},
                          
                         '购买金额_事件表':{'mean':30000,'std':15000},'数量_事件表':{'mean':4,'std':2},
                         '可用余额_事件表':{'mean':50000,'std':20000},'分期金额_事件表':{'mean':15000,'std':9000},
                         '贷款期限_事件表':{'mean':14,'std':4},'未还本金_事件表':{'mean':13000,'std':5000},
                         '偿还金额_事件表':{'mean':9000,'std':3000},'金额_事件表':{'mean':6000,'std':3000},
                         '交易金额_交易日志表':{'mean':2500,'std':900},'产品万份收益_基金产品':{'mean':0.7,'std':0.2},  
                         '车产价值_字典列表':{'mean':50000,'std':7000},'房产价值_字典列表':{'mean':300000,'std':80000},
                         '交易金额_交易往来':{'mean':5000,'std':1000},'存期_交易账务':{'mean':12,'std':5},
                         '交易金额_交易账务':{'mean':6000,'std':2000},
                         '行内总资产_资产负债':{'mean':70000,'std':15000},'行内总负债_资产负债':{'mean':12000,'std':5000},
                         '本外币存款_资产负债':{'mean':2000,'std':1000},'人民币活期存款_资产负债':{'mean':40000,'std':15000},
                         '人民币定期存款_资产负债':{'mean':20000,'std':9000},'本外币理财_资产负债':{'mean':900,'std':80},
                         '人民币理财_资产负债':{'mean':20000,'std':8000},'万利宝理财_资产负债':{'mean':10000,'std':4000},
                         '保本型理财_人民币_资产负债':{'mean':30000,'std':10000},'结构性存款_人民币_资产负债':{'mean':30000,'std':8000},
                         '兴业宝资产_资产负债':{'mean':15000,'std':5000},'综合金融资产_资产负债':{'mean':10000,'std':3000},
                         '券商集合理财_资产负债':{'mean':6000,'std':2000},'保险_资产负债':{'mean':1000,'std':300},
                         '个人贷款_资产负债':{'mean':50000,'std':5000},'个人购房按揭贷款_资产负债':{'mean':180000,'std':50000},
                         '个人委托贷款_资产负债':{'mean':8000,'std':2000},'个人消费贷款_资产负债':{'mean':4500,'std':9000},
                         '开户日期':{'mean':1000,'std':200},
                         '月收入_交易往来':{'mean':7000,'std':4000},
                         '月支出_交易往来':{'mean':2800,'std':1000},
                        }


true_result_df=create_column_randomsample(True_column_random_dict)
true_result_number_df=create_number_randomsample(True_number_random_dict)

True_combin_df=true_result_df.merge(true_result_number_df,left_index=True, right_index=True)
#True_combin_df=True_combin_df.merge(pd.DataFrame(sample_true[['id','owner_city']]),left_index=True, right_index=True)
#True_combin_df.rename(columns={'owner_city':'籍贯'},inplace=True)
True_combin_df['target']=1

False_result_df=create_column_randomsample(False_column_random_dict)
False_result_number_df=create_number_randomsample(False_number_random_dict)
False_combin_df=False_result_df.merge(False_result_number_df,left_index=True, right_index=True)
#False_combin_df=False_combin_df.merge(pd.DataFrame(sample_false[['id','owner_city']]),left_index=True, right_index=True)
#False_combin_df.rename(columns={'owner_city':'籍贯'},inplace=True)
False_combin_df['target']=0

total_comple=True_combin_df.append(False_combin_df).reset_index(drop=True)

#total_comple=total_comple[['id','性别','年龄','籍贯','婚姻状况','学历','征信认定年收入','社会保障','客户级信用额度','美元账户额度','CRM评级结果','客户评级结果','借记卡层级','附卡卡数','存款期限','是否为低保户','受否贫困户','行业','职业','零售潜在客户','理财产品风险等级','手机品牌','基金风险等级','理财产品是否保本','客户风险承受能力等级','电子渠道购买标志','贷款放款本金','账户余额','贷款期数','剩余贷款本金','购买金额_事件表','数量_事件表','可用余额_事件表','分期金额_事件表','贷款期限_事件表','未还本金_事件表','偿还金额_事件表','金额_事件表','交易金额_交易日志表','产品万份收益_基金产品','车产价值_字典列表','房产价值_字典列表','交易金额_交易往来','存期_交易账务','交易金额_交易账务','行内总资产_资产负债','行内总负债_资产负债','本外币存款_资产负债','人民币活期存款_资产负债','人民币定期存款_资产负债','本外币理财_资产负债','人民币理财_资产负债','万利宝理财_资产负债','保本型理财_人民币_资产负债','结构性存款_人民币_资产负债','兴业宝资产_资产负债','综合金融资产_资产负债','券商集合理财_资产负债','保险_资产负债','个人贷款_资产负债','个人购房按揭贷款_资产负债','个人委托贷款_资产负债','个人消费贷款_资产负债','target']]
total_comple=total_comple[['性别','年龄','婚姻状况','学历','开户日期','征信认定年收入','社会保障','客户级信用额度','美元账户额度','CRM评级结果','客户评级结果','借记卡层级','附卡卡数','存款期限','是否为低保户','受否贫困户','行业','职业','零售潜在客户','理财产品风险等级','手机品牌','基金风险等级','理财产品是否保本','客户风险承受能力等级','电子渠道购买标志','贷款放款本金','账户余额','贷款期数','剩余贷款本金','购买金额_事件表','数量_事件表','可用余额_事件表','分期金额_事件表','贷款期限_事件表','未还本金_事件表','偿还金额_事件表','金额_事件表','交易金额_交易日志表','产品万份收益_基金产品','车产价值_字典列表','房产价值_字典列表','交易金额_交易往来','存期_交易账务','交易金额_交易账务','行内总资产_资产负债','行内总负债_资产负债','本外币存款_资产负债','人民币活期存款_资产负债','人民币定期存款_资产负债','本外币理财_资产负债','人民币理财_资产负债','万利宝理财_资产负债','保本型理财_人民币_资产负债','结构性存款_人民币_资产负债','兴业宝资产_资产负债','综合金融资产_资产负债','券商集合理财_资产负债','保险_资产负债','个人贷款_资产负债','个人购房按揭贷款_资产负债','个人委托贷款_资产负债','个人消费贷款_资产负债','target']]


# 生成随机日期函数：
import datetime
def create_random_date(rowdata):
    #print (datetime.datetime.now()+datetime.timedelta(days=-1)).strftime("%Y-%m-%d %H:%M:%S")
    #print(datetime.datetime.now()+datetime.timedelta(days=rowdata['开户日期']).strftime("%Y-%m-%d"))
    print((datetime.datetime.now()-datetime.timedelta(days=rowdata['开户日期'])).strftime("%Y-%m-%d"))
    return (datetime.datetime.now()-datetime.timedelta(days=rowdata['开户日期'])).strftime("%Y-%m-%d")
total_comple['开户日期']=total_comple.apply(create_random_date,axis=1)

1999-08-15
2019-09-17
2015-03-27
2002-10-13
2003-02-18
2017-08-01
2007-06-27
2009-06-06
1990-07-08
2017-06-26
2000-03-04
2001-10-02
2006-01-21
2004-08-31
2010-02-28
2018-01-11
2004-04-14
2014-03-18
2011-11-10
2018-11-06
2003-01-08
2018-09-17
1990-06-21
2011-08-08
1987-11-27
2011-12-07
2019-09-29
2004-05-01
1997-09-03
2018-02-25
2014-01-01
2019-08-10
2010-10-01
2018-05-02
2015-05-15
2019-01-08
2004-09-10
1991-06-26
2015-03-26
1994-10-16
2005-09-21
2007-03-24
2015-06-30
2002-07-02
2016-08-21
2009-04-17
1994-01-27
1994-09-26
2008-09-05
2018-11-24
2011-01-23
2002-11-07
2013-12-17
2002-07-09
1990-02-09
1994-07-04
2017-05-24
2018-11-08
2017-06-17
1998-09-06
2012-06-16
1988-08-05
2018-12-03
2010-02-15
2000-11-01
2012-02-15
2012-08-07
1999-06-11
2004-02-13
1987-11-03
1984-11-03
2017-03-27
1981-08-21
1997-06-20
1991-06-26
2017-11-05
2014-04-25
2019-11-21
1999-01-16
2002-05-10
2005-05-29
2016-09-04
1993-07-05
2019-04-27
2015-04-16
2011-01-07
2008-03-21
2008-10-07
2017-05-19
1982-01-11
2014-05-19

1999-03-06
1997-04-06
1992-07-09
2006-02-11
2004-09-27
2005-12-08
1998-01-15
2002-12-29
2009-04-28
2008-10-13
2007-12-04
2013-07-04
2003-01-03
1995-07-22
2013-01-02
2019-05-20
2013-10-27
2014-05-17
2008-12-24
2010-09-12
2003-04-08
1989-09-10
2001-01-18
2010-08-30
2012-11-02
2014-01-13
2002-10-01
2013-05-13
1996-07-18
2004-11-25
2011-02-19
2002-06-14
2006-02-22
1991-08-01
2018-06-14
2015-12-07
2002-06-04
2008-04-09
2013-07-28
2013-05-09
2014-11-05
1992-12-31
2018-07-09
2014-12-27
2001-02-20
2010-05-12
2010-05-04
2018-10-15
2017-12-31
1996-05-18
2009-07-19
2006-05-09
1995-01-21
1995-05-05
2017-10-17
2004-05-24
2016-09-30
2019-11-24
2005-04-24
2009-01-22
2006-01-13
2012-04-23
2013-01-14
2013-09-18
2006-02-09
2010-11-08
1999-06-07
2001-05-09
2003-04-20
1994-06-12
2017-09-20
2005-10-29
1997-04-29
2017-11-29
2005-03-22
2011-03-19
2008-03-15
2004-02-26
2006-07-27
2015-04-22
2012-05-13
2014-04-20
2017-06-01
2000-05-17
2018-08-17
2007-02-04
2011-09-18
2010-09-04
2019-08-15
1996-03-05
2012-03-25

2013-05-26
2004-09-11
2018-06-30
2000-01-23
2009-03-28
2007-07-29
1997-02-22
2009-01-12
2003-08-16
2019-07-28
2019-12-08
2013-11-19
2013-08-31
2013-01-29
1994-11-15
2016-06-10
2019-03-30
2008-06-29
2015-11-14
2015-09-14
2018-12-22
1982-06-06
2008-08-20
1994-11-15
2017-05-13
2006-03-24
2016-09-28
2002-03-07
1987-11-18
2006-05-29
2014-12-12
2019-10-14
2012-02-04
2007-11-03
2000-06-16
2002-12-29
2013-12-06
2004-08-15
1999-09-25
2014-03-17
2013-01-15
2008-08-30
2010-04-01
1980-08-18
2018-06-23
2001-06-04
2012-01-22
2015-11-11
1997-04-01
2017-07-14
2005-10-25
2014-10-12
2015-02-20
2007-09-10
2001-04-03
2005-04-13
2002-04-16
2014-08-23
2010-05-09
2006-06-22
2000-09-09
2001-04-03
2004-12-28
1993-10-13
2009-08-18
2009-03-04
2004-10-11
2012-10-05
2018-06-27
2013-02-24
2008-07-18
2013-05-08
2001-11-12
2015-12-28
2013-07-04
2000-07-30
2011-07-03
2004-06-23
2004-03-30
2018-06-26
2013-02-15
1989-02-22
2018-02-09
2010-12-26
2014-02-22
1998-10-19
1990-12-04
2005-12-06
2006-06-22
2001-08-22
2006-04-25

2005-07-10
2017-12-09
2006-12-19
2007-10-11
2008-05-17
2010-09-30
1997-01-10
2012-12-31
2019-04-12
2019-12-09
2000-10-24
2018-12-05
2001-04-26
2004-05-27
2016-09-04
2016-12-28
2000-10-01
2003-04-05
2013-03-10
1999-10-21
1990-01-29
2017-12-05
2010-10-21
2017-08-13
2002-07-23
2007-11-27
2001-06-25
2007-04-25
2002-09-27
1984-08-03
2010-06-15
2013-08-24
2007-07-19
2006-11-19
2008-08-23
2012-09-27
2015-09-26
2001-07-16
2005-09-23
2004-12-10
2018-01-18
2019-11-18
2009-01-20
2013-10-06
2014-07-29
2016-01-06
2000-01-02
2008-07-13
2009-01-03
2009-01-27
2004-08-22
2004-06-05
2002-09-04
2002-06-29
1997-03-30
2013-10-12
2004-08-02
2001-07-12
2005-01-02
2018-10-17
1998-07-06
1987-08-09
2015-01-20
2019-04-02
2006-05-17
2012-03-26
1999-09-04
2011-04-13
1993-08-17
2009-11-16
2002-01-18
2010-05-12
2019-09-29
2004-02-21
2012-07-22
2008-06-23
2003-07-10
2015-02-02
2016-11-30
2002-09-16
2007-05-25
2016-05-24
2014-04-12
2011-01-05
2003-03-13
2008-06-10
2017-03-08
2018-05-31
2008-01-29
1996-02-05
2012-01-13

1993-12-20
2014-01-13
2002-05-05
2016-09-10
2003-09-02
2006-12-25
2016-12-20
2011-03-28
2008-10-14
1995-09-22
2018-05-22
2016-10-29
2004-06-18
2007-12-06
2007-12-12
2001-09-30
2005-10-15
2001-11-09
2013-01-14
2014-12-03
2015-05-11
2014-01-28
2006-04-19
2008-12-25
2011-11-17
1999-05-27
2009-01-24
2018-07-26
2003-12-19
2004-07-26
2017-07-17
2013-12-22
1997-11-17
2019-04-09
2004-04-29
2014-05-04
1987-12-08
1997-01-13
2006-09-04
2019-10-09
2014-06-03
2017-12-15
1997-11-10
2016-11-15
2002-08-05
2004-02-12
2016-05-18
2009-06-29
2002-08-29
2014-09-11
2012-12-21
2013-04-28
2015-06-23
2014-01-14
2019-02-13
2002-04-17
1996-09-29
2016-10-05
2017-08-13
2009-12-09
2007-11-20
2016-06-18
2019-01-09
2018-05-30
2012-09-01
1996-12-25
2010-08-18
1998-12-02
2018-03-08
2014-01-07
2014-10-02
2000-09-27
2005-03-15
2008-02-18
2003-06-04
2013-01-23
2001-12-10
2007-08-03
2010-10-22
2009-02-03
2009-10-13
1983-09-12
2011-10-17
2006-11-08
1996-06-25
2007-09-09
1989-12-15
2006-08-05
1998-07-24
2011-09-23
1999-03-25

...

2017-05-14
2017-05-05
2017-09-09
2016-04-06
2017-07-25
2017-09-09
2017-02-20
2016-07-16
2016-11-25
2017-07-08
2016-09-28
2018-06-29
2018-06-07
2017-12-13
2017-07-26
2016-08-28
2016-10-28
2016-06-15
2016-10-21
2017-08-15
2017-04-28
2017-12-05
2017-02-02
2017-06-20
2016-10-21
2017-04-02
2017-10-01
2017-07-09
2016-09-30
2017-06-08
2018-01-25
2017-06-01
2017-05-22
2017-04-23
2016-11-04
2017-05-13
2016-12-21
2017-03-09
2017-04-24
2018-03-22
2017-06-17
2017-08-20
2016-10-01
2017-11-11
2017-10-26
2016-11-04
2017-04-09
2017-07-07
2016-11-03
2016-12-10
2016-01-02
2017-09-29
2017-04-14
2017-05-22
2017-04-30
2016-12-09
2017-08-19
2016-12-12
2015-10-13
2017-06-29
2016-12-14
2016-12-04
2017-11-02
2017-03-02
2018-03-25
2017-07-20
2016-06-17
2017-07-01
2017-05-15
2017-02-16
2017-11-08
2017-07-28
2017-04-03
2016-10-23
2016-05-27
2016-04-02
2017-12-22
2017-01-26
2017-09-23
2017-04-13
2017-03-30
2017-03-05
2017-05-12
2017-10-16
2017-09-23
2017-03-27
2016-11-22
2017-10-19
2016-04-27
2016-05-22
2017-01-17

2016-04-17
2017-10-11
2017-01-01
2017-05-07
2017-04-17
2017-01-12
2017-01-01
2016-03-24
2016-01-09
2017-01-16
2016-10-01
2017-07-18
2016-07-15
2017-11-14
2016-10-29
2018-06-14
2017-04-15
2017-09-28
2016-08-14
2017-04-16
2017-09-08
2017-11-08
2016-04-19
2017-12-21
2017-09-05
2017-12-08
2017-07-07
2016-02-28
2016-12-13
2017-07-22
2016-10-23
2017-07-13
2017-11-13
2018-09-20
2016-08-27
2016-11-21
2017-02-22
2017-06-21
2017-04-14
2016-09-19
2017-07-05
2017-04-09
2016-02-12
2017-05-21
2017-07-19
2017-01-04
2016-12-31
2017-08-27
2017-07-10
2017-05-05
2017-03-28
2016-12-23
2017-06-30
2017-03-04
2017-06-14
2017-02-24
2017-08-20
2016-12-22
2017-01-09
2016-12-09
2017-06-28
2017-03-24
2016-09-14
2016-12-31
2017-04-14
2017-05-14
2017-10-06
2017-03-01
2017-01-08
2016-12-29
2017-06-27
2016-11-11
2017-03-28
2017-01-22
2017-01-09
2017-09-17
2017-01-17
2017-04-12
2018-02-27
2016-08-21
2017-07-11
2017-08-04
2017-04-09
2016-09-30
2017-08-07
2017-03-08
2017-01-02
2017-09-25
2016-05-31
2016-09-17
2017-08-15

2018-02-08
2017-11-02
2017-08-08
2017-08-16
2017-03-27
2018-05-27
2017-02-28
2016-12-18
2017-12-25
2017-02-12
2017-03-27
2017-07-03
2017-06-07
2017-04-27
2018-01-22
2017-11-16
2016-07-22
2017-05-17
2015-04-25
2018-01-07
2017-03-02
2017-11-18
2017-08-21
2017-05-21
2017-11-26
2018-07-10
2018-06-27
2017-11-14
2017-02-07
2017-12-05
2017-11-22
2018-04-11
2017-08-08
2017-08-24
2017-04-16
2017-03-26
2017-12-31
2016-03-01
2015-07-14
2016-09-19
2017-07-02
2017-06-08
2017-11-11
2016-06-11
2016-09-29
2017-06-20
2017-11-01
2016-09-24
2017-01-08
2016-09-22
2017-07-06
2017-04-11
2016-09-27
2018-03-03
2017-11-28
2017-02-14
2017-11-12
2016-12-17
2017-06-03
2017-04-06
2017-03-01
2017-07-04
2016-12-07
2017-09-14
2017-10-17
2016-10-29
2016-03-29
2017-03-15
2018-04-16
2017-10-27
2017-09-04
2016-09-20
2017-09-09
2017-12-22
2016-05-10
2016-05-06
2017-01-20
2017-06-08
2017-02-10
2017-09-11
2017-01-20
2016-08-04
2017-01-28
2017-08-11
2016-10-08
2017-08-05
2017-05-13
2017-10-23
2017-09-09
2017-07-04
2017-06-15

2017-10-31
2017-07-08
2017-12-21
2017-12-14
2017-11-04
2017-03-01
2017-10-06
2016-09-15
2016-05-11
2018-05-01
2018-02-23
2016-11-14
2017-01-03
2017-07-09
2017-04-01
2016-10-12
2017-04-13
2016-08-01
2017-10-25
2017-08-24
2017-07-05
2017-04-27
2016-12-15
2016-07-28
2018-02-16
2017-05-17
2016-10-31
2017-05-04
2017-09-07
2017-04-29
2016-07-02
2017-01-24
2017-01-26
2017-08-01
2017-06-22
2016-06-20
2017-04-11
2015-12-30
2017-11-26
2017-12-22
2016-12-28
2017-10-01
2017-02-05
2017-01-14
2017-06-04
2017-09-16
2016-11-19
2015-12-27
2017-06-27
2017-06-29
2017-09-18
2017-10-12
2016-08-31
2017-03-14
2017-05-06
2016-09-26
2015-12-04
2017-02-16
2017-12-25
2016-10-05
2016-01-10
2017-04-06
2016-09-04
2016-10-01
2016-07-16
2016-12-25
2017-11-11
2017-07-03
2017-07-17
2016-12-11
2017-10-17
2016-04-19
2016-11-17
2017-01-13
2016-04-18
2016-08-04
2017-12-09
2017-12-26
2017-09-09
2016-07-30
2016-12-30
2017-06-22
2016-08-05
2017-07-13
2016-09-15
2018-02-10
2017-09-27
2017-04-03
2016-11-27
2018-03-23
2017-12-28

2016-11-18
2018-05-21
2017-08-03
2016-08-06
2016-12-18
2017-03-04
2018-02-01
2017-03-30
2017-10-11
2017-03-30
2016-12-19
2017-05-14


In [135]:
total_comple.head()

,性别,年龄,婚姻状况,学历,开户日期,征信认定年收入,社会保障,客户级信用额度,美元账户额度,CRM评级结果,...,结构性存款_人民币_资产负债,兴业宝资产_资产负债,综合金融资产_资产负债,券商集合理财_资产负债,保险_资产负债,个人贷款_资产负债,个人购房按揭贷款_资产负债,个人委托贷款_资产负债,个人消费贷款_资产负债,target
0,男,12,未婚,大专,1999-08-15,5万,0,20000,0,核心,...,49405,23432,25468,10041,4184,78639,376452,12277,5605,1
1,女,25,已婚,初中中专,2019-09-17,10万,1,50000,0,普通,...,39884,18805,14512,7880,3957,80815,379467,11764,3858,1
2,男,-10,未婚,大专,2015-03-27,5万,0,10000,0,普通,...,50171,11834,11639,9109,4451,66963,295913,18028,5434,1
3,男,20,已婚,大专,2002-10-13,10万,1,30000,0,核心,...,44882,19946,9732,10432,4529,70032,510295,14891,4267,1
4,男,19,已婚,大专,2003-02-18,5万,0,50000,0,普通,...,44677,15478,12109,9432,4324,72221,343757,18678,5915,1


In [136]:
# 对样本数据进行处理，将负值修改为0

random_columns_list=['贷款放款本金','账户余额','贷款期数','剩余贷款本金','购买金额_事件表','数量_事件表','可用余额_事件表','分期金额_事件表','贷款期限_事件表','未还本金_事件表','偿还金额_事件表','金额_事件表','交易金额_交易日志表','产品万份收益_基金产品','车产价值_字典列表','房产价值_字典列表','交易金额_交易往来','存期_交易账务','交易金额_交易账务','行内总资产_资产负债','行内总负债_资产负债','本外币存款_资产负债','人民币活期存款_资产负债','人民币定期存款_资产负债','本外币理财_资产负债','人民币理财_资产负债','万利宝理财_资产负债','保本型理财_人民币_资产负债','结构性存款_人民币_资产负债','兴业宝资产_资产负债','综合金融资产_资产负债','券商集合理财_资产负债','保险_资产负债','个人贷款_资产负债','个人购房按揭贷款_资产负债','个人委托贷款_资产负债','个人消费贷款_资产负债']


def deal_zero_value(rowdata,column):
    if rowdata[column]<0:
        return 0
    else:
        return rowdata[column]
    
    
for random_colum in random_columns_list:    
    total_comple[random_colum]=total_comple.apply(deal_zero_value,axis=1, args = (random_colum,))


In [137]:
display(True_combin_df.head(),False_combin_df.head(),total_comple.tail())
print(total_comple.shape,True_combin_df.shape,False_combin_df.shape,total_comple.columns)

,性别,婚姻状况,学历,征信认定年收入,社会保障,客户级信用额度,美元账户额度,CRM评级结果,客户评级结果,借记卡层级,...,兴业宝资产_资产负债,综合金融资产_资产负债,券商集合理财_资产负债,保险_资产负债,个人贷款_资产负债,个人购房按揭贷款_资产负债,个人委托贷款_资产负债,个人消费贷款_资产负债,开户日期,target
0,男,未婚,大专,5万,0,20000,0,核心,C5,金卡,...,23432,25468,10041,4184,78639,376452,12277,5605,7446.714355,1
1,女,已婚,初中中专,10万,1,50000,0,普通,C1,金卡,...,18805,14512,7880,3957,80815,379467,11764,3858,108.866684,1
2,男,未婚,大专,5万,0,10000,0,普通,C3,普卡,...,11834,11639,9109,4451,66963,295913,18028,5434,1744.237549,1
3,男,已婚,大专,10万,1,30000,0,核心,C2,白金,...,19946,9732,10432,4529,70032,510295,14891,4267,6292.313477,1
4,男,已婚,大专,5万,0,50000,0,普通,C4,普卡,...,15478,12109,9432,4324,72221,343757,18678,5915,6164.348145,1


,性别,婚姻状况,学历,征信认定年收入,社会保障,客户级信用额度,美元账户额度,CRM评级结果,客户评级结果,借记卡层级,...,兴业宝资产_资产负债,综合金融资产_资产负债,券商集合理财_资产负债,保险_资产负债,个人贷款_资产负债,个人购房按揭贷款_资产负债,个人委托贷款_资产负债,个人消费贷款_资产负债,开户日期,target
0,男,已婚,小学,40万,1,10000,0,普通,C3,普卡,...,11747,9048,9531,929,47535,213117,6595,22222,1088.429199,0
1,女,未婚,大专,5万,1,10000,0,准核心,C2,白金,...,7289,12950,8118,861,48017,162023,11036,9673,1220.908081,0
2,女,已婚,高中中专,5万,0,20000,0,普通,C4,普卡,...,15520,9893,7520,767,53450,151895,10159,-3269,957.995117,0
3,男,未婚,高中中专,5万,1,10000,0,普通,C3,普卡,...,8076,9410,4284,487,45945,103304,6625,-11558,1077.512817,0
4,女,未婚,高中中专,5万,1,10000,0,普通,C3,白金,...,13992,13881,4240,786,55331,176267,10152,-4469,689.687500,0


,性别,年龄,婚姻状况,学历,开户日期,征信认定年收入,社会保障,客户级信用额度,美元账户额度,CRM评级结果,...,结构性存款_人民币_资产负债,兴业宝资产_资产负债,综合金融资产_资产负债,券商集合理财_资产负债,保险_资产负债,个人贷款_资产负债,个人购房按揭贷款_资产负债,个人委托贷款_资产负债,个人消费贷款_资产负债,target
99995,男,29,已婚,高中中专,2017-03-30,10万,1,20000,0,普通,...,41915,15396,9325,5536,906,50827,82965,11304,9481,0
99996,男,37,未婚,高中,2017-10-11,10万,1,5000,0,核心,...,25032,24468,12483,5174,1037,48866,183879,8211,0,0
99997,男,25,未婚,高中中专,2017-03-30,10万,0,10000,0,普通,...,27005,18522,9303,6758,721,54899,182742,10197,8090,0
99998,女,29,未婚,高中中专,2016-12-19,10万,1,50000,0,普通,...,45068,12262,5797,4672,1036,52021,243886,9382,613,0
99999,男,9,未婚,大专,2017-05-14,5万,0,5000,0,准核心,...,35195,13049,12545,4444,1359,47716,170418,9210,19285,0


(100000, 63) (50000, 63) (50000, 63) Index(['性别', '年龄', '婚姻状况', '学历', '开户日期', '征信认定年收入', '社会保障', '客户级信用额度',
       '美元账户额度', 'CRM评级结果', '客户评级结果', '借记卡层级', '附卡卡数', '存款期限', '是否为低保户',
       '受否贫困户', '行业', '职业', '零售潜在客户', '理财产品风险等级', '手机品牌', '基金风险等级', '理财产品是否保本',
       '客户风险承受能力等级', '电子渠道购买标志', '贷款放款本金', '账户余额', '贷款期数', '剩余贷款本金',
       '购买金额_事件表', '数量_事件表', '可用余额_事件表', '分期金额_事件表', '贷款期限_事件表', '未还本金_事件表',
       '偿还金额_事件表', '金额_事件表', '交易金额_交易日志表', '产品万份收益_基金产品', '车产价值_字典列表',
       '房产价值_字典列表', '交易金额_交易往来', '存期_交易账务', '交易金额_交易账务', '行内总资产_资产负债',
       '行内总负债_资产负债', '本外币存款_资产负债', '人民币活期存款_资产负债', '人民币定期存款_资产负债',
       '本外币理财_资产负债', '人民币理财_资产负债', '万利宝理财_资产负债', '保本型理财_人民币_资产负债',
       '结构性存款_人民币_资产负债', '兴业宝资产_资产负债', '综合金融资产_资产负债', '券商集合理财_资产负债', '保险_资产负债',
       '个人贷款_资产负债', '个人购房按揭贷款_资产负债', '个人委托贷款_资产负债', '个人消费贷款_资产负债', 'target'],
      dtype='object')


In [138]:
total_comple.to_excel('d:/上海兴业银行样本数据集.xlsx',index=None)


In [ ]:
# 生成随机日期函数：
import
def create_random_date(rowdata):
    print(rowdata.)
    

In [8]:
True_combin_df.head()

,性别,婚姻状况,学历,征信认定年收入,社会保障,客户级信用额度,美元账户额度,CRM评级结果,客户评级结果,借记卡层级,...,存款期限,是否为低保户,受否贫困户,年龄,贷款放款本金,账户余额,贷款期数,剩余贷款本金,id,籍贯
0,女,未婚,本科,5万,1,20000,小于0,核心,C4,普卡,...,6个月,否,否,10,539739,-7114,123,404910,13013302938,联通联通
1,男,已婚,大专,20万,1,20000,大于0,核心,C4,金卡,...,1年,否,否,48,807197,74019,63,404393,13022062073,NaN
2,男,已婚,高中中专,5万,1,10000,小于0,核心,C6,白金,...,1年,否,否,11,679981,216705,117,63766,13035901978,NaN
3,男,已婚,高中,5万,1,50000,小于0,核心,C1,白金,...,5年,否,否,30,498424,185155,201,330451,13042151031,NaN
4,男,已婚,大专,10万,1,50000,小于0,核心,C1,普卡,...,5年,否,否,12,511549,115947,-6,326409,13045173577,NaN


In [21]:
combin_df[['id','性别','籍贯','婚姻状况','学历','征信认定年收入','社会保障','客户级信用额度','美元账户额度','CRM评级结果','客户评级结果','借记卡层级','账户余额','剩余贷款本金']].to_excel('d:\上海兴业银行样本造数.xlsx',index=None)

In [164]:
display(true_result_df.head(),true_result_number_df.head())

,性别,婚姻状况,学历,征信认定年收入,社会保障,客户级信用额度,美元账户额度,CRM评级结果,客户评级结果,借记卡层级,附卡卡数,存款期限,是否为低保户,受否贫困户
0,男,已婚,大专,5万,1,30000,小于0,普通,C3,黑金,4,1年,否,否
1,男,已婚,高中,10万,1,30000,小于0,准核心,C3,普卡,0,3个月,否,否
2,女,已婚,本科,5万,0,20000,小于0,普通,C4,普卡,3,1年,否,否
3,男,已婚,大专,10万,1,20000,小于0,普通,C3,金卡,1,2年,否,否
4,女,已婚,小学,5万,1,20000,小于0,普通,C3,白金,0,5年,否,否


,年龄,贷款放款本金,账户余额,贷款期数,剩余贷款本金
0,37,691320,8,317,445082
1,15,645061,29,49,196353
2,19,579867,29,157,368844
3,10,667989,34,33,329207
4,30,566405,51,118,330431


In [12]:
pd.DataFrame

,id,target,card_type,netlong,owner_city,arpu,out_prdct_fee,prdct_fee,point_fee,myth_fee,...,age,adis,arpu_avg,myth_fee_avg,out_prdct_fee_avg,prdct_fee_avg,os,battery_capacity,storage,screen_size
97032,18746191488,1,1.0,37.0,黑龙江哈尔滨,45.32,6.48,29.0,0.0,0.0,...,NaN,1.0,45.320000,0.0,6.480000,29.000000,NaN,NaN,NaN,NaN
29750,13769713610,1,0.0,NaN,云南曲靖,115.70,0.00,0.0,0.0,0.0,...,NaN,NaN,115.099998,0.0,NaN,0.000000,4.0,2920mAH,16GB/32GB,5.0
36394,13838040676,1,1.0,NaN,河南郑州,143.20,0.20,135.0,9951.0,0.0,...,NaN,1.0,148.810003,0.0,0.200000,135.000000,4.0,3000mAh,32GB,5.5
79629,15945389676,1,0.0,30.0,黑龙江牡丹江,60.99,1.00,90.0,0.0,0.0,...,NaN,NaN,60.990002,0.0,1.000000,90.000000,4.0,3000mAh,16GB,5.7
40228,13888246777,1,1.0,NaN,云南昆明,258.00,0.00,0.0,0.0,0.0,...,NaN,NaN,258.000000,0.0,NaN,0.000000,3.0,1715mAh,16GB/64GB/128GB,4.7
82230,15980726225,1,1.0,NaN,福建福州,78.00,0.00,62.0,0.0,0.0,...,35.0,NaN,91.333333,0.0,NaN,75.333333,4.0,2300mAh,16GB,5.0
62550,15217163366,1,2.0,50.0,广东珠海,55.12,0.00,28.0,0.0,0.0,...,NaN,NaN,54.293334,0.0,NaN,28.000000,4.0,3100mAh,8GB,5.5
99664,18811186422,1,1.0,43.0,北京北京,118.00,0.00,118.0,0.0,0.0,...,NaN,NaN,118.224998,0.0,NaN,118.000000,4.0,3000mAh,16GB,5.5
93368,18374069022,1,0.0,44.0,湖南株洲,31.49,0.00,0.0,0.0,0.0,...,NaN,NaN,22.496667,0.0,NaN,0.000000,NaN,NaN,NaN,NaN
13231,13580829297,1,0.0,NaN,广东东莞,178.00,0.00,173.0,0.0,0.0,...,NaN,NaN,176.866669,0.0,NaN,171.333333,4.0,3750mAh,64GB/128GB/256GB,5.5
